# Custom FunctionNode with invocation_state
Deterministic 함수를 Graph 노드로 사용하며 invocation_state로 데이터 공유

In [ ]:
from strands.multiagent import GraphBuilder
from strands.multiagent.base import MultiAgentBase, NodeResult, Status, MultiAgentResult
from strands.agent.agent_result import AgentResult
from strands.types.content import ContentBlock, Message

## 1. Custom FunctionNode 정의

## 2. invocation_state를 사용하는 함수들

In [ ]:
class FunctionNode(MultiAgentBase):
    """Deterministic Python 함수를 Graph 노드로 실행 (invocation_state 지원)"""

    def __init__(self, func, name: str = None):
        super().__init__()
        self.func = func
        self.name = name or func.__name__

    async def invoke_async(self, task, invocation_state=None, **kwargs):
        # 함수 실행 (invocation_state 전달)
        result = self.func(
            task if isinstance(task, str) else str(task),
            invocation_state
        )

        agent_result = AgentResult(
            stop_reason="end_turn",
            message=Message(role="assistant", content=[ContentBlock(text=str(result))]),
            input_tokens=0,
            output_tokens=0,
            total_tokens=0
        )

        return MultiAgentResult(
            status=Status.COMPLETED,
            results={self.name: NodeResult(
                result=agent_result,
                status=Status.COMPLETED,
                execution_time=0
            )},
            execution_order=[],
            total_nodes=1,
            completed_nodes=1,
            failed_nodes=0,
            execution_time=0,
            invocation_state=invocation_state
        )

In [ ]:
def collect_entities(text: str, invocation_state: dict) -> str:
    """엔티티 추출 및 invocation_state에 저장"""
    # 간단한 엔티티 추출 (대문자로 시작하는 단어)
    words = text.split()
    entities = [w for w in words if w[0].isupper() and len(w) > 2]
    
    collected = invocation_state.get("entities", [])
    collected.extend(entities)
    invocation_state["entities"] = collected
    
    print(f"✓ Collected entities: {entities}")
    return f"Collected {len(entities)} entities"

def collect_relations(text: str, invocation_state: dict) -> str:
    """관계 추출 및 invocation_state에 저장"""
    relations = invocation_state.get("relations", [])
    
    # 간단한 관계 추출
    if "integrates with" in text:
        parts = text.split("integrates with")
        if len(parts) == 2:
            source = parts[0].strip().split()[-1]
            target = parts[1].strip().split()[0]
            
            relations.append({
                "source": source,
                "target": target,
                "relation": "integrates_with"
            })
            invocation_state["relations"] = relations
            
            print(f"✓ Collected relation: {source} -> {target}")
            return f"Collected 1 relation"
    
    return "No relations found"

def summarize_graph(text: str, invocation_state: dict) -> str:
    """invocation_state에서 수집된 그래프 데이터 요약"""
    entities = invocation_state.get("entities", [])
    relations = invocation_state.get("relations", [])
    
    summary = f"\n=== Graph Summary ===\n"
    summary += f"Entities ({len(entities)}): {entities}\n"
    summary += f"Relations ({len(relations)}): {relations}"
    
    print(summary)
    return summary

## 3. FunctionNode 생성 및 Graph 구성

In [ ]:
# FunctionNode 인스턴스 생성
entity_collector = FunctionNode(func=collect_entities, name="entity_collector")
relation_collector = FunctionNode(func=collect_relations, name="relation_collector")
summarizer = FunctionNode(func=summarize_graph, name="summarizer")

# Graph 구성
builder = GraphBuilder()

builder.add_node(entity_collector, "collect_entities")
builder.add_node(relation_collector, "collect_relations")
builder.add_node(summarizer, "summarize")

builder.add_edge("collect_entities", "collect_relations")
builder.add_edge("collect_relations", "summarize")

builder.set_entry_point("collect_entities")

graph = builder.build()

## 4. Graph 실행

In [ ]:
result = graph(
    "AWS Lambda integrates with Amazon S3 and DynamoDB stores data.",
    invocation_state={"entities": [], "relations": []}
)

print("\n=== Final Result ===")
print(f"Status: {result.status}")
print(f"Execution order: {[node.node_id for node in result.execution_order]}")
print(f"\nFinal State:")
print(f"Entities: {result.invocation_state.get('entities')}")
print(f"Relations: {result.invocation_state.get('relations')}")

## 구조 설명

```
Graph (invocation_state 초기화)
  ↓
collect_entities (FunctionNode)
  ↓ collect_entities(text, invocation_state)
  → invocation_state["entities"] 수정
  ↓
collect_relations (FunctionNode)
  ↓ collect_relations(text, invocation_state)
  → invocation_state["relations"] 수정
  ↓
summarize (FunctionNode)
  ↓ summarize_graph(text, invocation_state)
  → invocation_state 조회 및 요약
```

**장점:**
- ✅ Deterministic 로직으로 빠른 실행
- ✅ LLM 호출 없이 비즈니스 로직 처리
- ✅ invocation_state로 노드 간 데이터 공유
- ✅ Agent와 FunctionNode 혼합 사용 가능